In [1]:
import time
from sim_worker_threads import Simulations, SimUtils

SIM_PATH = r"..\mass-sim\simulator.js"
SECRET_PATH = r"client_secret_85915911832-bchaso494se2t02b45m3c670v56gjccn.apps.googleusercontent.com.json"

MASS_SIM_SHEET_NAME = "Beast/Aether/Avian Guild War" # Cloud's public sheet
OPEN_SIM_SHEET_NAME = "Spellstone Open Mass Sim"

max_threads = 3 # Number of CPU threads minus 1
bge = "178,179" # BGE TODO: auto-detect bge from data.min.json
siege = True # whether the opponent has any tower. default True (e.g. bounties)
tower = "501" # the opponent's tower type. default "501" (e.g. bounties)
N = 30 # minimum time interval between updates in seconds

In [2]:
sim = Simulations(SIM_PATH, max_threads, bge, siege, tower)
open_sim = SimUtils.SheetUtils(SECRET_PATH, OPEN_SIM_SHEET_NAME)

In [3]:
full_attackers, full_defenders = open_sim.get_hash_stash()

In [4]:
# mass_sheet = SimUtils.SheetUtils(SECRET_PATH, MASS_SIM_SHEET_NAME)
# full_attackers, full_defenders = mass_sheet.get_cloud_hashes()

In [5]:
# attackers, defenders, off_notes, def_notes = open_sim.get_open_sim_hashes()

# if len(attackers) > 0:
#     off_results = sim.mass_sim(attackers, full_defenders, num_sims=2000, include_top50=True)
#     open_sim.update_open_sim_results(off_results.iloc[2:, :2].reindex(attackers).assign(**{"Notes/Names": off_notes}), "off", True)

# if len(defenders) > 0:
#     def_results = sim.mass_sim(full_attackers, defenders, num_sims=2000, include_top50=True)
#     open_sim.update_open_sim_results(def_results.T.iloc[2:, :2].reindex(defenders).assign(**{"Notes/Names": def_notes}), "def", True)

In [6]:
# infinite loop
try:
    while True:
        ## core block
        start_time = time.time()

        attackers, defenders, off_notes, def_notes = open_sim.get_open_sim_hashes()

        if len(attackers) > 0:
            off_results = sim.mass_sim(attackers, full_defenders, num_sims=1000, include_top50=True)
            open_sim.update_open_sim_results(off_results.iloc[2:, :2].reindex(attackers).assign(**{"Notes/Names": off_notes}), "off", fifo=True)

        if len(defenders) > 0:
            def_results = sim.mass_sim(full_attackers, defenders, num_sims=1000, include_top50=True)
            open_sim.update_open_sim_results(def_results.T.iloc[2:, :2].reindex(defenders).assign(**{"Notes/Names": def_notes}), "def", fifo=True)

        print(f"Execution time: {time.time() - start_time:.2f} seconds  ", end="\r")        

        # Wait until the next N-second interval
        current_time = time.localtime()
        # N = 15
        seconds_until_next_interval = N - (current_time.tm_sec % N)
        time.sleep(seconds_until_next_interval)

except KeyboardInterrupt:
    print("Execution stopped by user     ")